In [2]:
from google.colab import files
upload=files.upload()

Saving Data_for_UCI_named.csv to Data_for_UCI_named.csv


In [72]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('Data_for_UCI_named.csv')
df.shape

(10000, 14)

In [5]:
df.columns

Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4', 'stab', 'stabf'],
      dtype='object')

In [71]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [6]:
df1 = df.drop('stab', axis = 1)

In [8]:
X = df1.drop('stabf', axis = 1)
y= df1.stabf

In [9]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.2 , random_state= 1 )
y_train.value_counts()

unstable    5092
stable      2908
Name: stabf, dtype: int64

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)

# 1.Using the ExtraTreesClassifier as your estimator with cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1 and random_state = 1. What are the best hyperparameters from the randomized search CV?

In [40]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(ExtraTreesClassifier( n_jobs = -1, verbose = 1 , random_state = 1), { 
    'min_samples_leaf': [4,6,8],
    "min_samples_split": [2,5,7],
   'n_estimators': [100,300,500,1000],
   'max_features': [None, 'log2', "auto"]
    
},cv=5, return_train_score = False, n_iter = 10)
                     
rs.fit(x_train,y_train)
rs.best_params_,rs.best_score_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent work

({'n_estimators': 100,
  'min_samples_split': 7,
  'min_samples_leaf': 8,
  'max_features': None},
 0.9237500000000001)

# 2. What is the accuracy on the test set using the XGboost classifier? In 4 decimal places.

In [42]:
# Fitting XGBoost to the training data
# encode string class values as integers
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y_train)
label_encoded_y = label_encoder.transform(y_train)
my_model = xgb.XGBClassifier()
my_model.fit(x_train_scale, label_encoded_y)
y_test_en  = label_encoder.transform(y_test)
print('The XGboost classifier accuracy =',my_model.score(x_test_scale,y_test_en))

The XGboost classifier accuracy = 0.9195


# 3. What is the accuracy on the test set using the random forest classifier? In 4 decimal places.

Options

In [44]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc = rfc.fit(x_train_scale,y_train)
print('The Random Forest Accuracy = ', rfc.score(x_test_scale,y_test))

The Random Forest Accuracy =  0.9225


# 4. What is the accuracy on the test set using the LGBM classifier? In 4 decimal places.

In [45]:
# Build the lightgbm model
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(x_train_scale, y_train)
print("The accuaracy of LGBM is",clf.score(x_test_scale,y_test))

The accuaracy of LGBM is 0.9375


# 5 Find the feature importance using the optimal ExtraTreesClassifier model. Which features are the most and least important respectively?

In [64]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(X, y)
value =pd.DataFrame(model.feature_importances_,index = X.columns)
value.sort_values(by=0,ascending= False)

,0
tau2,0.119212
tau1,0.118190
tau4,0.116470
tau3,0.114692
g3,0.099084
g2,0.092900
g4,0.091288
g1,0.089507
p2,0.040117
p4,0.039790


Tau2 and p1 are the most and least important respectively

# 6. Train a new ExtraTreesClassifier Model with the new Hyperparameters from the RandomizedSearchCV (with random_state = 1). Is the accuracy of the new optimal model higher or lower than the initial ExtraTreesClassifier model with no hyperparameter tuning?

In [66]:
# No hyperparameter
model = ExtraTreesClassifier()
model.fit(x_train_scale,y_train)
model.score(x_test_scale, y_test)

0.9315

In [70]:
# hyperparameter from Randomsearch cv.
ext = ExtraTreesClassifier(n_estimators= 100, min_samples_split= 7, min_samples_leaf=8,max_features= None)
ext.fit(x_train_scale,y_train)
ext.score(x_test_scale, y_test)

0.9245

It is lower